In [2]:
!pip install transformers[torch] datasets 

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
!pip install gdown -U --no-cache-dir

In [40]:
import gdown

url = 'https://drive.google.com/drive/folders/111D3XhuqDeoK3Yh0zBcs1yQIlRdJ3R-E?usp=sharing'

gdown.download_folder(url,remaining_ok=True) 

Retrieving folder list


Processing file 1_QhKFwPMa9lC2w0fQ0mw1QhuapCmXwge 108-upanishads-with-upanishad-brahmam-commentary.txt
Processing file 17AASH3Si_WAtt4frmHW49JGE0EWAWTx3 1318_sri-ramchritmanas_roman.txt
Processing file 19BFJARsEvXtBKjYIXbRv6YbVIPWf6ke9 2015.499647.The-Sushruta_text.txt
Processing file 1IgVGBopYbZYLEOGrVbuFuQ-yzSzDV48t abhidhamma.txt
Processing file 1z4P263Rm9hB-bWI4Q4FRSLXJFovz5C-R abhidhaultsci.txt
Processing file 1V5gYfodFODeBznOVTwjtw8Ojv9KzyCzU acariya-mun-bio.txt
Processing file 14u1angGO_g6D5q7RocZM9VAxuiXP5nTx acariya-mun.txt
Processing file 1UMad9QWHJE2wnlJ7wy_kUFbt_LY3su9O Agni Purana.txt
Processing file 1GbG5uwm236XOlWadeyjkGJ9VjjQW6WcZ allexistence.txt
Processing file 1UDwuoFqROJQqiCkE-Tj5JBO2xIRbjBy5 arthuravalonkularnavatantraeng.txt
Processing file 1WTolAVfoXLEF-5xRtk3ENgTCuewyXzWs aryabhata_with_english_commentary.txt
Processing file 1jcXtjLbU7aoHHlXxVUXyhkifsLRgZyNa atharvaveda.txt
Processing file 1xD3E6-4P5P2hju-LhHKi4yp6EZlIHE5G Bhagavadmaha Purana_djvu.txt
Processing

Retrieving folder list completed
Building directory structure


Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1_QhKFwPMa9lC2w0fQ0mw1QhuapCmXwge 

Download ended unsuccessfully


In [28]:
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM, EvalPrediction
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer,TrainingArguments
import os
import numpy as np

In [9]:
model=AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer=AutoTokenizer.from_pretrained("gpt2")

In [32]:
data_dir="/kaggle/working/train"
all_txt=''
for file_name in os.listdir(data_dir):
    file_path=os.path.join(data_dir,file_name)
    if file_path.endswith(".txt"):
        with open(file_path,"r",encoding="utf-8") as file:
            all_txt+=file.read()

In [33]:
combined_file_path = "/kaggle/working/file.txt"
with open(combined_file_path, "w", encoding="utf-8") as file:
    file.write(all_txt)

In [34]:
len(all_txt)

33127424

In [19]:
tokenizer.use_gpu=True
tokenizer

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)

In [35]:
dataset=TextDataset(
tokenizer=tokenizer,
file_path=combined_file_path,
block_size=128)

/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [21]:
dataset

In [22]:
data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False)

In [23]:
def compute_metrics(p: EvalPrediction):
    logits = p.predictions
    labels = p.label_ids
    probabilities = softmax(logits, axis=-1)
    loss = log_loss(labels.flatten(), probabilities.reshape(-1, probabilities.shape[-1]), labels=[i for i in range(logits.shape[-1])])
    perplexity = np.exp(loss)
    return {"perplexity": perplexity}

In [36]:
training_args = TrainingArguments(
    output_dir="gpt2-spiritual",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    learning_rate=1e-4,
    save_total_limit=2,
    push_to_hub=True,
    report_to='tensorboard',
    load_best_model_at_end=True,
    save_strategy="epoch",
    metric_for_best_model='eval_perplexity',
    evaluation_strategy='epoch',    
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [37]:
trainer.train()
#trainer.push_to_hub()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.push_to_hub()